In [1]:
import os

from sklearn.model_selection import train_test_split
import matplotlib
import argparse
import random
import numpy as np
import csv
import pandas as pd
from datascience import *
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import Model
from keras.models import Sequential, load_model
from keras.layers import *
from keras.optimizers import Adam
from keras import Input
from keras.preprocessing.text import Tokenizer
from keras.utils import plot_model
import keras
from keras.backend.tensorflow_backend import set_session
#config = tf.ConfigProto()
# config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
# config.gpu_options.per_process_gpu_memory_fraction = 0.7
#config.gpu_options.allow_growth =True

#set_session(tf.Session(config=config)) 
import re
os.environ['PYTHONHASHSEED'] = '0'
random.seed(1)
np.random.seed(1)
tf.set_random_seed(1)
from keras.callbacks import *

Using TensorFlow backend.


In [4]:
#parsing of the quora dataset

counter = 0
list1 = []
with open("quora_duplicate_questions.tsv", 'r', encoding='utf-8') as fd:
  
    for x in fd:
        columns = x.split("\t")
               
       
        list1.append(columns)

print(len(list1))

        

404301


In [27]:
print(list1[1][3], list1[1][4])
print(levenshtein(list1[1][3],list1[1][4]))


#let's use levenshtein as a baseline model for the quora dataset
dists=[None for i in range (len(list1)-1)]

vector = [0]*2
all_dist = [0]*(len(list1)-1)
all_bool = [0]*(len(list1)-1)

for x in range (1, 1000):
    
    
    vector[0] = (levenshtein(list1[x][3], (list1[x][4])))
    all_dist[x]=vector[0]
    
    vector[1] = (sanitize(list1[x][5]))
    all_bool[x]=vector[1]
    
    dists[x-1]=vector

distances = all_dist.values
booleans = all_bool.values


#training
dist_train, dist_test, bool_train, bool_test = train_test_split(all_dist, all_bool, test_size=0.25, random_state=1000)
 


classifier = LogisticRegression()
classifier.fit(dist_train, bool_train)

#testing

score = classifier.score(dist_test, bool_test)
print(score)

What is the step by step guide to invest in share market in india? What is the step by step guide to invest in share market?
9


AttributeError: 'list' object has no attribute 'values'

In [8]:
def sanitize(text):
    # Strip \n
  
    text = text.replace('\n', '')
 
 
    return text

print(sanitize("is_duplicate\n"))

is_duplicate


In [6]:
#first thots on alg: Levenshtein distance

#feature vector would thus be in this format: (dist, boolean)


#number of same words

#feature vector would thus be: (int, boolean)



def levenshtein(s1, s2):   #https://en.wikibooks.org/wiki/Algorithm_Implementation/Strings/Levenshtein_distance#Python
    if len(s1) < len(s2):
        return levenshtein(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]


In [31]:
train_tb=Table.read_table("dataset/Phrase_Level/PAWS_QQP/LABELED_PAWS_QQP_TRAIN.tsv")

In [32]:
id_col=train_tb.apply(int, "TRAIN_PAWS_QQP_id")
PPDB_id = train_tb.apply(int, "PPDB_id")

In [36]:
phrase_dataset = train_tb.with_columns("TRAIN_PAWS_QQP_id", id_col, "PPDB_id", PPDB_id, "Ground Label", llist)

In [37]:
phrase_dataset.show(4)

TRAIN_PAWS_QQP_id,PPDB_id,sent1,sent2,paraphrase1,paraphrase2,labels1,labels2,Ground Label
4,44326,b'What are the most common traffic convictions in Arizon ...,b'What are the most common traffic convictions in Arkans ...,most common,common,O O O B E O O O O O O O O O O O O O O O O O,O O O O S O O O O O O O O O O O O O O O O O,0
10,103288,b'Which are in your opinion the most beautiful and the m ...,b'Which are in your opinion the most depressing and the ...,most beautiful,beautiful,O O O O O O B E O O O O O O O O,O O O O O O O O O O O S O O O O,1
14,316010,b'Is nuclear energy renewable energy ?',"b""`` Is nuclear energy `` renewable energy `` ? ''""",nuclear energy,nuclear,O B E O O O,O O S O O O O O O O,1
18,526060,b'What ancient units would definitely win in a 1v1 fight ...,b'What numerous modern units would definitely win in a 1 ...,fight against,fight,O O O O O O O O O B E O O O O,O O O O O O O O O O S O O O O,0


In [39]:
phrase_dataset=phrase_dataset.to_df()
phrase_dataset.to_csv("dataset/Phrase_Level/PAWS_QQP/LABELED_PAWS_QQP_TRAIN.tsv", sep='\t')

In [40]:
dev_tb=Table.read_table("dataset/Phrase_Level/PAWS_QQP/LABELED_PAWS_QQP_DEV.tsv")

In [47]:
id_col=dev_tb.apply(int, "DEV_PAWS_QQP_id")
PPDB_id = dev_tb.apply(int, "PPDB_id")
phrase_dataset = dev_tb.with_columns("DEV_PAWS_QQP_id", id_col, "PPDB_id", PPDB_id, "Ground Label", llist)
phrase_dataset.show(4)

DEV_PAWS_QQP_id,PPDB_id,sent1,sent2,paraphrase1,paraphrase2,labels1,labels2,Ground Label
3,84905,"b""Why ca n't countries afford high quality products , bu ...","b""Why ca n't countries afford China 's high quality prod ...",high quality,quality,O O O O O B E O O O O O O O O O O O O O O O O O O O,O O O O O O O O S O O O O O O O O O O O O O O O O O O O,0
3,129607,"b""Why ca n't countries afford high quality products , bu ...","b""Why ca n't countries afford China 's high quality prod ...",high quality,quality,O O O O O B E O O O O O O O O O O O O O O O O O O O,O O O O O O O O S O O O O O O O O O O O O O O O O O O O,0
11,72492,b'How do I start learning electronic music production ? ...,b'How do I begin learning electronic music production ? ...,begin with,start with,O O O O O O O O O O O B E O O O O O O O O O O O O O O O ...,O O O O O O O O O O O B E O O O O O O O O O O O O O O O ...,1
11,366448,b'How do I start learning electronic music production ? ...,b'How do I begin learning electronic music production ? ...,begin with,start with,O O O O O O O O O O O B E O O O O O O O O O O O O O O O ...,O O O O O O O O O O O B E O O O O O O O O O O O O O O O ...,1


In [48]:
phrase_dataset=phrase_dataset.to_df()
phrase_dataset.to_csv("dataset/Phrase_Level/PAWS_QQP/LABELED_PAWS_QQP_DEV.tsv", sep='\t')

In [41]:
list1=[]
list2=[]
Tb=Table()
with open("dataset/Phrase_Level/PAWS_QQP/LABELED_PAWS_QQP_DEV.tsv", 'r', encoding='utf-8') as fd:
    
    rd = csv.reader(fd, delimiter="\t", quotechar='"')
    for row in rd:
        
        list1.append(row) #pairs start at index 1, with id, setence1, sentence2, label, respectively.
     
    for y in range(8):
        temp=[]
        for x in range(1, len(list1)):
            temp.append(list1[x][y])
            
        list2.append(temp)

    for ele in range(len(list2)):
        Tb=Tb.with_columns(list1[0][ele], list2[ele])


Tb.show(50)

DEV_PAWS_QQP_id,PPDB_id,sent1,sent2,paraphrase1,paraphrase2,labels1,labels2
3.0,84905.0,"b""Why ca n't countries afford high quality products , bu ...","b""Why ca n't countries afford China 's high quality prod ...",high quality,quality,O O O O O B E O O O O O O O O O O O O O O O O O O O,O O O O O O O O S O O O O O O O O O O O O O O O O O O O
3.0,129607.0,"b""Why ca n't countries afford high quality products , bu ...","b""Why ca n't countries afford China 's high quality prod ...",high quality,quality,O O O O O B E O O O O O O O O O O O O O O O O O O O,O O O O O O O O S O O O O O O O O O O O O O O O O O O O
11.0,72492.0,b'How do I start learning electronic music production ? ...,b'How do I begin learning electronic music production ? ...,begin with,start with,O O O O O O O O O O O B E O O O O O O O O O O O O O O O ...,O O O O O O O O O O O B E O O O O O O O O O O O O O O O ...
11.0,366448.0,b'How do I start learning electronic music production ? ...,b'How do I begin learning electronic music production ? ...,begin with,start with,O O O O O O O O O O O B E O O O O O O O O O O O O O O O ...,O O O O O O O O O O O B E O O O O O O O O O O O O O O O ...
22.0,543619.0,"b""I 'm currently working for a Big 4 as an Advanced Tax ...","b""I 'm currently working for a Big 4 as an Advanced Tax ...",much better,much,O O O O O O O O O O O O O O O O O O O O O O O O O O O O ...,O O O O O O O O O O O O O O O O O O O O O O O O O O O O ...
25.0,487239.0,"b'In Nevada when pulling out of a parking lot , can you ...","b'In Nevada when pulling out of a parking lot , can you ...",far left,left,O O O O O O O O O O O O O O O O O O B E O O O O O O O O ...,O O O O O O O O O O O O O O S O O O O O O O O O O O O O ...
29.0,284980.0,"b""What are the reasons for SAARC 's failure as regional ...","b""What are the reasons for SAARC 's failure as common or ...",economic cooperation,cooperation,O O O O O O O O O O O O O O O B E O O O O O O O,O O O O O O O O O O O O O O O O O O O O O O S O
56.0,459668.0,"b'I want to be a child psychologist , what qualification ...","b'I want to become a child psychologist , what qualifica ...",become one,be,O O O O O O O O O O O O O O B E O O O O O O O O O O O O O,O O O O O O O O O O O O O O S O O O O O O O O O O O O O O
57.0,71262.0,b'Why are beautiful girls so rude all the time ? I saw a ...,b'Why are rude girls so beautiful all the time ? I saw a ...,very rude,rude,O O O O O O O O O O O O O O O O O O O O O O O O O O O B ...,O O S O O O O O O O O O O O O O O O O O O O O O O O O O ...
57.0,494710.0,b'Why are beautiful girls so rude all the time ? I saw a ...,b'Why are rude girls so beautiful all the time ? I saw a ...,go away,away,O O O O O O O O O O O O O O O O O O O O O O O O O O O O ...,O O O O O O O O O O O O O O O O O O O O O O O O O O O O ...


In [23]:
list1=[]
list2=[]
Tb_sup=Table()
with open("dataset/PAWS_QQP/PAWS_QQP_dev_and_test.tsv", 'r', encoding='utf-8') as fd:
    rd = csv.reader(fd, delimiter="\t", quotechar='"')
    for row in rd:
        
        list1.append(row) #pairs start at index 1, with id, setence1, sentence2, label, respectively.
     
    for y in range(4):
        temp=[]
        for x in range(1, len(list1)):
            temp.append(list1[x][y])
            
        list2.append(temp)

    for ele in range(len(list2)):
        Tb_sup=Tb_sup.with_columns(list1[0][ele], list2[ele])




In [43]:
Tb_sup=Table.read_table("dataset/PAWS_QQP/PAWS_QQP_dev_and_test.tsv")

In [44]:
ll=Tb_sup.column("label")
llist = []
for x in ((Tb.column(0))):
    p = float(x)
    llist=np.append(llist, (ll[int(p)]))  
    


In [45]:
print(llist[:51])


[0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 1. 1. 0.]
